Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy_, valamint a _scipy.stats t_ és _f_ osztályai

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path


In [3]:
datalib = Path.cwd().parent.joinpath('data')

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.




3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

In [7]:
import pandas as pd
data1 = pd.read_parquet('../data/sp500.parquet', engine='fastparquet')
data2 = pd.read_parquet('../data/ff_factors.parquet', engine='fastparquet')
merged = data1.merge(data2, on='Date', how='left')
merged['Excess Return'] = merged['Monthly Returns'] - merged['RF']
merged = merged.sort_index()
merged['ex_ret_1'] = merged.groupby('Symbol')['Excess Return'].shift(-1)
merged = merged.dropna(subset=['ex_ret_1'])
merged = merged.dropna(subset=['HML'])
merged

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF,Excess Return,ex_ret_1
Date,,,,,,,,
2000-01-31,CPRT,-0.987130,-0.0474,0.0577,-0.0188,0.041,-1.028130,0.194133
2000-01-31,LNT,-0.988305,-0.0474,0.0577,-0.0188,0.041,-1.029305,-0.099604
2000-01-31,BKNG,6.844284,-0.0474,0.0577,-0.0188,0.041,6.803284,-0.078560
2000-01-31,SPG,-0.889351,-0.0474,0.0577,-0.0188,0.041,-0.930351,-0.076494
2000-01-31,LMT,-0.971967,-0.0474,0.0577,-0.0188,0.041,-1.012967,-0.100433
...,...,...,...,...,...,...,...,...
2022-10-31,MAA,0.024170,0.0783,0.0006,0.0801,0.023,0.001170,0.018190
2022-10-31,CME,-0.021622,0.0783,0.0006,0.0801,0.023,-0.044622,-0.010535
2022-10-31,EBAY,0.082314,0.0783,0.0006,0.0801,0.023,0.059314,0.117324


### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot.
 <u>Ebben a blokkban önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályodat.</u>

7., Készíts egy új __LinearRegressionNP__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami OLS elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között. <u>__(numpy.linalg.lstsq() nem használható)__</u>

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítsd ki az osztályt egy __get_wald_test_result__ metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Wald: wald_value, p-value: p_value__, ahol az wald_value és p_value helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. A függvény bemenete során feltételezzük, hogy r minden eleme 0, R-t listák listája formában adjuk át. A Wald statisztika p értékét az előző feladathoz hasonlóan számold ki, de figyelj, hogy a <u>teszt 1 oldalú</u>.

12., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

In [ ]:
merged = merged[merged['Symbol'] == 'AMZN']
merged = merged.drop(columns=['Symbol'])